In [1]:
import sqlite3 
import pandas as pd 
import numpy as np 
import datetime 
import os 
import plotly.express as px
import sys

In [7]:
sys.path.append('/home/malcolm/WorkTransit/scripts/')
from WorkTransitLib import RawDirections as rd
rev_locations = {v:k for k, v in rd.locations.items()}
rd.locations

{'home': '120 Benchley Pl, Bronx NY',
 'work': '809 Washington Street',
 'bxbark': '830 Pelham Pkwy, Pelham Manor, NY 10803',
 'orchard_beach': 'Orchard Beach, Bronx, NY 10464'}

In [2]:
def create_con(db_path='/home/malcolm/WorkTransit/data/transit.db'):
    con = sqlite3.connect(db_path)
    cursor = con.cursor()
    cursor.execute('SELECT name FROM sqlite_master WHERE type=\'table\' ORDER BY name')
    tables = cursor.fetchall()
    tables = [x[0] for x in tables]
    print("Tables: ", tables)
    return(con, cursor)

In [3]:
con, cursor = create_con()

Tables:  ['Raw_Responses', 'parsed_driving', 'parsed_transit']


In [49]:
today = datetime.datetime.now()
today_str = str(today.date())
two_weeks = today - datetime.timedelta(days=21)
two_weeks_str = str(two_weeks.date())
print("Today: ", today_str, "Two weeks: ", two_weeks_str)

Today:  2021-11-27 Two weeks:  2021-11-06


# Driving

## Load Data

In [50]:
# Demo with Raw Data
df = pd.read_sql(f""" 
select * 
from Raw_Responses
where Date between '{two_weeks_str}' and '{today_str}'
order by Date desc
""", con)
print(df.shape)
df.head()

(1626, 7)


,origin,destination,mode,departure_time,Raw Response,Date,Time
0,"120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-27 07:05:18.415322,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-27,07:05
1,"Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-11-27 07:05:18.415322,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-11-27,07:05
2,"120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-27 07:19:56.508645,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-27,07:19
3,"Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-11-27 07:19:56.508645,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-11-27,07:19
4,"120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-27 07:35:10.901615,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-27,07:35


In [51]:
# Load Data with Modifications
df = pd.read_sql(f""" 
select * 
from parsed_driving
where Date between '{two_weeks_str}' and '{today_str}'
order by Date
""", con)
print(df.shape)
df['departure_time'] = pd.to_datetime(df['departure_time'])
df['origin_name'] = df['origin'].apply(lambda x: rev_locations.get(x, 'Unknown'))
df['destination_name'] = df['destination'].apply(lambda x: rev_locations.get(x, 'Unknown'))
df['Day of Week'] = df['departure_time'].dt.dayofweek.astype(str) + '-' + df['departure_time'].dt.day_name()
df.head()

(1392, 17)


,summary,duration_hr,duration_text,duration_in_traffic_hr,duration_in_traffic_text,extra time in traffic_min,distance_mi,1st Longest Step (time),2nd Longest Step (time),3rd Longest Step (time),origin,destination,mode,departure_time,Raw Response,Date,Time,origin_name,destination_name,Day of Week
0,Orchard Beach Rd,0.22,13 mins,0.19,11 mins,-1.8,3.83,"{'Step #': 5, 'distance_mi': 0.54, 'duration_h...","{'Step #': 3, 'distance_mi': 0.31, 'duration_h...","{'Step #': 13, 'distance_mi': 0.27, 'duration_...","120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-06 07:03:06.503305,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-06,07:03,home,orchard_beach,5-Saturday
1,City Island Rd,0.21,12 mins,0.17,10 mins,-2.4,3.84,"{'Step #': 13, 'distance_mi': 0.43, 'duration_...","{'Step #': 6, 'distance_mi': 0.68, 'duration_h...","{'Step #': 9, 'distance_mi': 0.54, 'duration_h...","Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-11-06 07:03:06.503305,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-11-06,07:03,orchard_beach,home,5-Saturday
2,Orchard Beach Rd,0.22,13 mins,0.20,12 mins,-1.2,3.83,"{'Step #': 5, 'distance_mi': 0.54, 'duration_h...","{'Step #': 3, 'distance_mi': 0.31, 'duration_h...","{'Step #': 13, 'distance_mi': 0.27, 'duration_...","120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-06 07:18:10.956772,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-06,07:18,home,orchard_beach,5-Saturday
3,City Island Rd,0.21,12 mins,0.17,10 mins,-2.4,3.84,"{'Step #': 13, 'distance_mi': 0.43, 'duration_...","{'Step #': 6, 'distance_mi': 0.68, 'duration_h...","{'Step #': 9, 'distance_mi': 0.54, 'duration_h...","Orchard Beach, Bronx, NY 10464","120 Benchley Pl, Bronx NY",driving,2021-11-06 07:18:10.956772,[{'bounds': {'northeast': {'lat': 40.874488100...,2021-11-06,07:18,orchard_beach,home,5-Saturday
4,Orchard Beach Rd,0.22,13 mins,0.20,12 mins,-1.2,3.83,"{'Step #': 5, 'distance_mi': 0.54, 'duration_h...","{'Step #': 3, 'distance_mi': 0.31, 'duration_h...","{'Step #': 13, 'distance_mi': 0.27, 'duration_...","120 Benchley Pl, Bronx NY","Orchard Beach, Bronx, NY 10464",driving,2021-11-06 07:33:09.736035,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-06,07:33,home,orchard_beach,5-Saturday


In [52]:
df['Day of Week'].value_counts()

4-Friday       338
1-Tuesday      218
3-Thursday     216
0-Monday       216
2-Wednesday    213
5-Saturday     110
6-Sunday        81
Name: Day of Week, dtype: int64

In [53]:
df.groupby(['Day of Week', 'origin_name', 'destination_name']).count()['summary']

Day of Week  origin_name    destination_name
0-Monday     bxbark         work                36
             home           bxbark              36
                            work                36
             work           bxbark              72
                            home                36
1-Tuesday    bxbark         work                36
             home           bxbark              36
                            orchard_beach        1
                            work                36
             orchard_beach  home                 1
             work           bxbark              72
                            home                36
2-Wednesday  bxbark         work                35
             home           bxbark              35
                            work                35
             work           bxbark              72
                            home                36
3-Thursday   bxbark         work                36
             home           bxbark   

In [54]:
df.groupby('Date').count()

,summary,duration_hr,duration_text,duration_in_traffic_hr,duration_in_traffic_text,extra time in traffic_min,distance_mi,1st Longest Step (time),2nd Longest Step (time),3rd Longest Step (time),origin,destination,mode,departure_time,Raw Response,Time,origin_name,destination_name,Day of Week
Date,,,,,,,,,,,,,,,,,,,
2021-11-06,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
2021-11-07,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
2021-11-08,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
2021-11-09,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
2021-11-10,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
2021-11-11,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
2021-11-12,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72,72
2021-11-13,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28
2021-11-14,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28


In [55]:
df.groupby(['Date', 'origin', 'destination']).count()

summary  \
Date       origin                                  destination                                        
2021-11-06 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                     16   
2021-11-07 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                     16   
2021-11-08 120 Benchley Pl, Bronx NY               809 Washington Street                         12   
...                                                                                             ...   
2021-11-26 809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803       48   
           830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                         22   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                     28   
2021-11-27 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                19   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                      8   

                                                                                            duration_hr  \
Date       origin                                  destination                                            
2021-11-06 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                    12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                         16   
2021-11-07 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                    12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                         16   
2021-11-08 120 Benchley Pl, Bronx NY               809 Washington Street                             12   
...                                                                                                 ...   
2021-11-26 809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803           48   
           830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                             22   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                         28   
2021-11-27 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                    19   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                          8   

                                                                                            duration_text  \
Date       origin                                  destination                                              
2021-11-06 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                      12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                           16   
2021-11-07 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                      12   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                           16   
2021-11-08 120 Benchley Pl, Bronx NY               809 Washington Street                               12   
...                                                                                                   ...   
2021-11-26 809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803             48   
           830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                               22   
           Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY                           28   
2021-11-27 120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464                      19   
           Orchard Beach, Bronx, NY 10464          1

In [56]:
df.groupby(['origin', 'destination', 'summary']).count().sort_values('duration_hr', ascending=False)

duration_hr  \
origin                                  destination                             summary                                  
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N                            374   
120 Benchley Pl, Bronx NY               830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                             189   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S                            164   
809 Washington Street                   120 Benchley Pl, Bronx NY               NY-9A N and I-95 N                 150   
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               City Island Rd                     129   
120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464          Orchard Beach Rd                   119   
                                        809 Washington Street                   NY-9A S                            112   
                                                                                I-95 S and NY-9A S                  61   
809 Washington Street                   120 Benchley Pl, Bronx NY               NY-9A N                             26   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   I-95 S and NY-9A S                  22   
120 Benchley Pl, Bronx NY               809 Washington Street                   FDR Dr                               9   
                                                                                NY-9A S/Henry Hudson Pkwy            7   
809 Washington Street                   120 Benchley Pl, Bronx NY               FDR Dr and I-278 E                   5   
                                                                                I-278 E                              4   
                                                                                I-95 N                               4   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S/Henry Hudson Pkwy            3   
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N/Henry Hudson Pkwy            2   
                                        120 Benchley Pl, Bronx NY               NY-9A N/Henry Hudson Pkwy            2   
                                        830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N and I-95 N                   2   
                                                                                I-678 N                              2   
                                                                                I-278 E                              2   
                                                                                FDR Dr and I-278 E                   2   
                                        120 Benchley Pl, Bronx NY               NY-9A N and I-87 N                   1   
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               Hutchinson River Pkwy S              1   

                                                                                                           duration_text  \
origin                                  destination                             summary                                    
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N                              374   
120 Benchley Pl, Bronx NY               830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                               189   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S                              164   
809 Washington Street                   120 Benchley Pl, Bronx NY               NY-9A N and I-95 N                   150   
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               City Island Rd                       129   
120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 1

In [57]:
def gb_summary_func(df):
    out = {}
    out['Count'] = df.shape[0]
    out['Avg Duration Traffic hr'] = np.round(df['duration_in_traffic_hr'].mean(), 2)
    out['Stdev Duration Traffic hr'] = np.round(df['duration_in_traffic_hr'].std(), 2)
    out['Avg Duration Traffic text'] = str(np.round(out['Avg Duration Traffic hr'] * 60, 2) ) + ' mins'
    out['Stdev Duration Traffic text'] = str(np.round(out['Stdev Duration Traffic hr'] * 60, 2) ) + ' mins'
    out['Avg Duration Mins num'] = np.round(out['Avg Duration Traffic hr'] * 60, 2)
    out['Stdev Duration Min num'] = np.round(out['Stdev Duration Traffic hr'] * 60, 2)
    out['Avg Duration Traffic Mins num'] = np.round(out['Avg Duration Traffic hr'] * 60, 2)
    out['Earliest Time'] = df['Time'].min()
    out['Latest Time'] = df['Time'].max()
    out['Earliest Date'] = df['Date'].min()
    out['Latest Date'] = df['Date'].max()
    out_series = pd.Series(out)
    return(out_series)

In [58]:
df.columns

Index(['summary', 'duration_hr', 'duration_text', 'duration_in_traffic_hr',
       'duration_in_traffic_text', 'extra time in traffic_min', 'distance_mi',
       '1st Longest Step (time)', '2nd Longest Step (time)',
       '3rd Longest Step (time)', 'origin', 'destination', 'mode',
       'departure_time', 'Raw Response', 'Date', 'Time', 'origin_name',
       'destination_name', 'Day of Week'],
      dtype='object')

In [59]:
origin_dest_cnts = df.groupby(['origin', 'destination'])['summary'].count()\
    .rename('Total Counts', axis=0)
origin_dest_cnts

origin                                   destination                            
120 Benchley Pl, Bronx NY                809 Washington Street                      189
                                         830 Pelham Pkwy, Pelham Manor, NY 10803    189
                                         Orchard Beach, Bronx, NY 10464             119
809 Washington Street                    120 Benchley Pl, Bronx NY                  192
                                         830 Pelham Pkwy, Pelham Manor, NY 10803    384
830 Pelham Pkwy, Pelham Manor, NY 10803  809 Washington Street                      189
Orchard Beach, Bronx, NY 10464           120 Benchley Pl, Bronx NY                  130
Name: Total Counts, dtype: int64

In [60]:
gb_summary = df.groupby(['origin', 'destination', 'summary']).apply(lambda x: gb_summary_func(x))
gb_summary2 = pd.merge(gb_summary, origin_dest_cnts
         , left_index=True, right_index=True )
gb_summary2['Summary %'] = np.round(100 * gb_summary2['Count']/gb_summary2['Total Counts'], 1)

gb_summary2[gb_summary2['Count'] >= 10]\
    .drop(['Avg Duration Traffic hr', 'Stdev Duration Traffic hr'], axis=1)\
    .sort_values(['origin', 'destination', 'Count'], ascending=False)

Count  \
origin                                  destination                             summary                     
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               City Island Rd        129   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S               164   
                                                                                I-95 S and NY-9A S     22   
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N               374   
                                        120 Benchley Pl, Bronx NY               NY-9A N and I-95 N    150   
                                                                                NY-9A N                26   
120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464          Orchard Beach Rd      119   
                                        830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                189   
                                        809 Washington Street                   NY-9A S               112   
                                                                                I-95 S and NY-9A S     61   

                                                                                                   Avg Duration Traffic text  \
origin                                  destination                             summary                                        
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               City Island Rd                     11.4 mins   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S                            52.2 mins   
                                                                                I-95 S and NY-9A S                 40.8 mins   
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N                            51.0 mins   
                                        120 Benchley Pl, Bronx NY               NY-9A N and I-95 N                 55.8 mins   
                                                                                NY-9A N                            63.0 mins   
120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464          Orchard Beach Rd                   12.0 mins   
                                        830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                              9.6 mins   
                                        809 Washington Street                   NY-9A S                            58.2 mins   
                                                                                I-95 S and NY-9A S                 49.2 mins   

                                                                                                   Stdev Duration Traffic text  \
origin                                  destination                             summary                                          
Orchard Beach, Bronx, NY 10464          120 Benchley Pl, Bronx NY               City Island Rd                        1.2 mins   
830 Pelham Pkwy, Pelham Manor, NY 10803 809 Washington Street                   NY-9A S                              11.4 mins   
                                                                                I-95 S and NY-9A S                   13.8 mins   
809 Washington Street                   830 Pelham Pkwy, Pelham Manor, NY 10803 NY-9A N                              14.4 mins   
                                        120 Benchley Pl, Bronx NY               NY-9A N and I-95 N                   16.2 mins   
                                                                                NY-9A N                              15.0 mins   
120 Benchley Pl, Bronx NY               Orchard Beach, Bronx, NY 10464          Orchard Beach Rd                      0.6 mins   
                                        830 Pelham Pkwy, Pelham Manor, NY 10803 US-1

In [61]:
gb_summary2 = pd.merge(gb_summary, origin_dest_cnts
         , left_index=True, right_index=True )
gb_summary2['Summary %'] = np.round(100 * gb_summary2['Count']/gb_summary2['Total Counts'], 1)
gb_summary2.head()

Count  \
origin                    destination                             summary                            
120 Benchley Pl, Bronx NY 809 Washington Street                   FDR Dr                         9   
                                                                  I-95 S and NY-9A S            61   
                                                                  NY-9A S                      112   
                                                                  NY-9A S/Henry Hudson Pkwy      7   
                          830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                       189   

                                                                                             Avg Duration Traffic hr  \
origin                    destination                             summary                                              
120 Benchley Pl, Bronx NY 809 Washington Street                   FDR Dr                                        1.16   
                                                                  I-95 S and NY-9A S                            0.82   
                                                                  NY-9A S                                       0.97   
                                                                  NY-9A S/Henry Hudson Pkwy                     0.99   
                          830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                                        0.16   

                                                                                             Stdev Duration Traffic hr  \
origin                    destination                             summary                                                
120 Benchley Pl, Bronx NY 809 Washington Street                   FDR Dr                                          0.08   
                                                                  I-95 S and NY-9A S                              0.26   
                                                                  NY-9A S                                         0.14   
                                                                  NY-9A S/Henry Hudson Pkwy                       0.10   
                          830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                                          0.01   

                                                                                            Avg Duration Traffic text  \
origin                    destination                             summary                                               
120 Benchley Pl, Bronx NY 809 Washington Street                   FDR Dr                                    69.6 mins   
                                                                  I-95 S and NY-9A S                        49.2 mins   
                                                                  NY-9A S                                   58.2 mins   
                                                                  NY-9A S/Henry Hudson Pkwy                 59.4 mins   
                          830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                                     9.6 mins   

                                                                                            Stdev Duration Traffic text  \
origin                    destination                             summary                                                 
120 Benchley Pl, Bronx NY 809 Washington Street                   FDR Dr                                       4.8 mins   
                                                                  I-95 S and NY-9A S                          15.6 mins   
                                                                  NY-9A S                                      8.4 mins   
                                                                  NY-9A S/Henry Hudson Pkwy                    6.0 mins   
                          830 Pelham Pkwy, Pelham Manor, NY 10803 US-1 N                        

## One Route 

Analyses requested.... by route, by day, by time 
metrics - average time to destination, min, max, (# of trips used in measurements) 

Overall fastest times, overall slowest times....

By trip (source and destination) 

In [62]:
origin1 = '120 Benchley Pl, Bronx NY'
dest1 = '809 Washington Street'
# day1 = df['Date'].max()

In [63]:
origin_name = route1['origin_name'].value_counts().index[0]
dest_name = route1['destination_name'].value_counts().index[0]
routename = f"{origin_name} to {dest_name}"
print(routename)

home to work


In [64]:
route1 = df[(df['origin'] == origin1) 
            & (df['destination'] == dest1)  
#             & (df['Date'] == day1)
           ]
print("Shape: ", route1.shape)
route1

Shape:  (189, 20)


,summary,duration_hr,duration_text,duration_in_traffic_hr,duration_in_traffic_text,extra time in traffic_min,distance_mi,1st Longest Step (time),2nd Longest Step (time),3rd Longest Step (time),origin,destination,mode,departure_time,Raw Response,Date,Time,origin_name,destination_name,Day of Week
56,I-95 S and NY-9A S,0.77,46 mins,0.95,57 mins,10.8,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-08 07:03:06.359198,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-08,07:03,home,work,0-Monday
59,I-95 S and NY-9A S,0.77,46 mins,1.08,1 hour 5 mins,18.6,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-08 07:18:10.742894,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-08,07:18,home,work,0-Monday
62,I-95 S and NY-9A S,0.82,49 mins,1.08,1 hour 5 mins,15.6,18.60,"{'Step #': 14, 'distance_mi': 8.58, 'duration_...","{'Step #': 12, 'distance_mi': 4.11, 'duration_...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-08 07:33:10.490875,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-08,07:33,home,work,0-Monday
65,I-95 S and NY-9A S,0.86,52 mins,1.10,1 hour 6 mins,14.4,18.64,"{'Step #': 19, 'distance_mi': 8.58, 'duration_...","{'Step #': 17, 'distance_mi': 3.54, 'duration_...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-08 07:48:09.602747,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-08,07:48,home,work,0-Monday
68,FDR Dr,0.85,51 mins,1.16,1 hour 10 mins,18.6,22.30,"{'Step #': 13, 'distance_mi': 8.67, 'duration_...","{'Step #': 8, 'distance_mi': 4.73, 'duration_h...","{'Step #': 16, 'distance_mi': 2.51, 'duration_...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-08 08:04:36.318711,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-08,08:04,home,work,0-Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,I-95 S and NY-9A S,0.77,46 mins,0.60,36 mins,-10.2,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-26 09:23:52.387142,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-26,09:23,home,work,4-Friday
1281,I-95 S and NY-9A S,0.77,46 mins,0.59,35 mins,-10.8,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-26 09:33:48.287454,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-26,09:33,home,work,4-Friday
1284,I-95 S and NY-9A S,0.77,46 mins,0.59,35 mins,-10.8,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-26 09:33:48.287454,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-26,09:33,home,work,4-Friday
1287,I-95 S and NY-9A S,0.77,46 mins,0.63,38 mins,-8.4,18.56,"{'Step #': 10, 'distance_mi': 8.58, 'duration_...","{'Step #': 8, 'distance_mi': 5.37, 'duration_h...","{'Step #': 6, 'distance_mi': 2.08, 'duration_h...","120 Benchley Pl, Bronx NY",809 Washington Street,driving,2021-11-26 09:48:19.179237,"[{'bounds': {'northeast': {'lat': 40.8745759, ...",2021-11-26,09:48,home,work,4-Friday


## By Day of Week and Route Graphs

In [65]:
route2 = route1.copy()
route2['departure_time_time'] = route2['departure_time'].apply(lambda x: x - pd.Timestamp(x.date()))

time_gb = route2.set_index('departure_time_time')\
    .groupby(['Day of Week', pd.Grouper(freq='20T')])\
    .apply(lambda x: gb_summary_func(x))
print("Destination: ", dest1, " Origin: ", origin1)
time_gb2 = time_gb.reset_index()
time_gb2['Earliest Time2'] = pd.to_datetime(time_gb2['Earliest Time'])

time_gb

Destination:  809 Washington Street  Origin:  120 Benchley Pl, Bronx NY


Count  Avg Duration Traffic hr  \
Day of Week departure_time_time                                      
0-Monday    0 days 07:03:06.359198      6                     1.10   
            0 days 07:23:06.359198      3                     1.16   
            0 days 07:43:06.359198      3                     1.17   
            0 days 08:03:06.359198      6                     1.16   
            0 days 08:23:06.359198      3                     1.11   
            0 days 08:43:06.359198      4                     1.07   
            0 days 09:03:06.359198      5                     0.93   
            0 days 09:23:06.359198      3                     0.82   
            0 days 09:43:06.359198      3                     0.83   
1-Tuesday   0 days 07:03:06.359198      6                     1.06   
            0 days 07:23:06.359198      3                     1.18   
            0 days 07:43:06.359198      3                     1.21   
            0 days 08:03:06.359198      5                     1.13   
            0 days 08:23:06.359198      4                     1.11   
            0 days 08:43:06.359198      3                     1.03   
            0 days 09:03:06.359198      6                     0.95   
            0 days 09:23:06.359198      3                     0.81   
            0 days 09:43:06.359198      3                     0.80   
2-Wednesday 0 days 07:03:06.359198      5                     1.06   
            0 days 07:23:06.359198      3                     1.14   
            0 days 07:43:06.359198      3                     1.15   
            0 days 08:03:06.359198      6                     1.12   
            0 days 08:23:06.359198      3                     1.06   
            0 days 08:43:06.359198      3                     1.02   
            0 days 09:03:06.359198      6                     0.92   
            0 days 09:23:06.359198      3                     0.85   
            0 days 09:43:06.359198      3                     0.85   
3-Thursday  0 days 07:03:06.359198      6                     0.81   
            0 days 07:23:06.359198      3                     0.88   
            0 days 07:43:06.359198      3                     0.88   
            0 days 08:03:06.359198      5                     0.81   
            0 days 08:23:06.359198      4                     0.93   
            0 days 08:43:06.359198      3                     0.82   
            0 days 09:03:06.359198      5                     0.75   
            0 days 09:23:06.359198      4                     0.76   
            0 days 09:43:06.359198      3                     0.75   
4-Friday    0 days 07:03:06.359198      6                     0.82   
            0 days 07:23:06.359198      4                     0.79   
            0 days 07:43:06.359198      4                     0.77   
            0 days 08:03:06.359198      8                     0.80   
            0 days 08:23:06.359198      4                     0.82   
            0 days 08:43:06.359198      5                     0.81   
            0 days 09:03:06.359198      5                     0.74   
            0 days 09:23:06.359198      6                     0.67   
            0 days 09:43:06.359198      4                     0.72   

                                    Stdev Duration Traffic hr  \
Day of Week departure_time_time                                 
0-Monday    0 days 07:03:06.359198                       0.10   
            0 days 07:23:06.359198                       0.09   
            0 days 07:43:06.359198                       0.07   
            0 days 08:03:06.359198                       0.04   
            0 days 08:23:06.359198                       0.05   
            0 days 08:43:06.359198                       0.07   
            0 days 09:03:06.359198                       0.08   
            0 days 09:23:06.359198                       0.04   
            0 days 09:43:06.359198                       0.05   
1-Tuesday   0 days 07:03:06.35

In [66]:
summary_gb = route2.set_index('departure_time_time')\
    .groupby(['summary', pd.Grouper(freq='20T')])\
    .apply(lambda x: gb_summary_func(x))
print("Destination: ", dest1, " Origin: ", origin1)

summary_gb2 = summary_gb.reset_index()
summary_gb2['Earliest Time2'] = pd.to_datetime(summary_gb2['Earliest Time'])
summary_gb

Destination:  809 Washington Street  Origin:  120 Benchley Pl, Bronx NY


Count  \
summary                   departure_time_time             
FDR Dr                    0 days 07:43:06.359198      3   
                          0 days 08:03:06.359198      2   
                          0 days 08:43:06.359198      2   
                          0 days 09:03:06.359198      2   
I-95 S and NY-9A S        0 days 07:03:06.359198      9   
                          0 days 07:23:06.359198      7   
                          0 days 07:43:06.359198      7   
                          0 days 08:03:06.359198     14   
                          0 days 08:23:06.359198      4   
                          0 days 08:43:06.359198      5   
                          0 days 09:03:06.359198      5   
                          0 days 09:23:06.359198      6   
                          0 days 09:43:06.359198      4   
NY-9A S                   0 days 07:03:06.359198     19   
                          0 days 07:23:06.359198      9   
                          0 days 07:43:06.359198      5   
                          0 days 08:03:06.359198     13   
                          0 days 08:23:06.359198     13   
                          0 days 08:43:06.359198     10   
                          0 days 09:03:06.359198     19   
                          0 days 09:23:06.359198     12   
                          0 days 09:43:06.359198     12   
NY-9A S/Henry Hudson Pkwy 0 days 07:03:06.359198      1   
                          0 days 07:43:06.359198      1   
                          0 days 08:03:06.359198      1   
                          0 days 08:23:06.359198      1   
                          0 days 08:43:06.359198      1   
                          0 days 09:03:06.359198      1   
                          0 days 09:23:06.359198      1   

                                                  Avg Duration Traffic hr  \
summary                   departure_time_time                               
FDR Dr                    0 days 07:43:06.359198                     1.22   
                          0 days 08:03:06.359198                     1.16   
                          0 days 08:43:06.359198                     1.12   
                          0 days 09:03:06.359198                     1.09   
I-95 S and NY-9A S        0 days 07:03:06.359198                     0.85   
                          0 days 07:23:06.359198                     0.92   
                          0 days 07:43:06.359198                     0.94   
                          0 days 08:03:06.359198                     0.90   
                          0 days 08:23:06.359198                     0.71   
                          0 days 08:43:06.359198                     0.76   
                          0 days 09:03:06.359198                     0.68   
                          0 days 09:23:06.359198                     0.64   
                          0 days 09:43:06.359198                     0.68   
NY-9A S                   0 days 07:03:06.359198                     1.01   
                          0 days 07:23:06.359198                     1.09   
                          0 days 07:43:06.359198                     1.02   
                          0 days 08:03:06.359198                     1.06   
                          0 days 08:23:06.359198                     1.07   
                          0 days 08:43:06.359198                     1.00   
                          0 days 09:03:06.359198                     0.89   
                          0 days 09:23:06.359198                     0.82   
                          0 days 09:43:06.359198                     0.82   
NY-9A S/Henry Hudson Pkwy 0 days 07:03:06.359198                     1.11   
                          0 days 07:43:06.359198                     0.96   
                          0 days 08:03:06.359198                     0.94   
                          0 days 08:23:06.359198                     1.16   
                          0 days 08:43:06.359198                

In [67]:
px.line(time_gb2, 'Earliest Time2', 'Avg Duration Mins num'
       , color='Day of Week'
       , title = "Avg Time to dest by Leave Time by Day of Week - " + routename)

In [68]:
px.line(summary_gb2, 'Earliest Time2', 'Avg Duration Mins num'
       , color='summary'
       , title = "Avg Time to dest by Leave Time by Route - " + routename)

## Metrics

In [69]:
df_summ = gb_summary_func(route1)
pd.DataFrame(df_summ).T\
    .drop(['Avg Duration Traffic hr', 'Stdev Duration Traffic hr', 
          'Avg Duration Mins num', 'Stdev Duration Min num', 'Avg Duration Traffic Mins num'], axis=1)


,Count,Avg Duration Traffic text,Stdev Duration Traffic text,Earliest Time,Latest Time,Earliest Date,Latest Date
0,189,55.8 mins,12.0 mins,07:03,09:54,2021-11-08,2021-11-26


# Transit 